# ADA Project

Now that we understand the dataset, we are going to try to answer our research questions in this notebook.

In [1]:
import pandas as pd
import time
from collections import deque
import numpy as np
pd.options.mode.chained_assignment = None

In [2]:
# read the dataframes
all_games = pd.read_pickle("data/games.pkl")
all_orders = pd.read_pickle("data/orders.pkl")
all_players = pd.read_pickle("data/players.pkl")
all_turns = pd.read_pickle("data/turns.pkl")
all_units = pd.read_pickle("data/units.pkl")

# remove duplicates
all_units = all_units.drop_duplicates()

# Descriptive statistics

We look at the **outcome** (binary) considering the **treatment**, which can be:
- engaged: player who was engaged in a friendship
- single: player who was not engaged in a friendship

And for engaged player, there is an aditional treatment: 
- betrayer: player who betrayed another one
- betrayed: player who ended up betrayed by another
- neutral: player who was not engaged in a *broken* friendship

In order to answer this question, let's define a few functions that we will use in the later analysis.

In [3]:
countries = ['A', 'E', 'F', 'G', 'I', 'R', 'T']
pairs = [x+y for x in countries for y in countries if y > x]

In [57]:
def get_engageds(friendships):
    """Returns the players engaged in a friendship"""
    cols = [col for col in friendships.columns if np.count_nonzero(friendships[col] != 0)]
    return list(set(''.join(cols)))

def get_singles(engageds):
    """Returns players not engaged in a friendship"""
    singles = countries.copy()
    for engaged in engageds:
        singles.remove(engaged)
    return singles

In [35]:
def get_betrayers_and_betrayed(friendships):
    """Given the Friendships dataframe as defined in our analysis, returns all the players who commited 
    betrayals and all players who ended up betrayed"""
    cols = [col for col in friendships.columns if np.count_nonzero(friendships[col] != 0)]
    betrayers = []
    betrayeds = []
    for c in cols: 
        tmp = friendships[c]
        values = tmp[tmp != 0].values
        if type(values[-1]) == str: 
            betrayer = values[-1]
            betrayers.append(betrayer)
            tmp = list(c)
            tmp.remove(betrayer)
            betrayeds.append(tmp[0])
            
    return betrayers, betrayeds

def get_neutrals(betrayers, betrayeds):
    """Given betrayers and betrayeds players of a game, returns the list of players
    who were not involved in a broken friendship
    
    Required: player must be 'engaged'
    """
    neutrals = countries.copy()
    for b in betrayers: 
        if b in neutrals: neutrals.remove(b)
    for b in betrayeds: 
        if b in neutrals: neutrals.remove(b)
    return neutrals

In [36]:
def get_winners(game_id):
    winner = all_players.query("game_id == @game_id & won == 1")
    return winner.country.values

def get_losers(winners):
    loosers = countries.copy()
    for w in winners: loosers.remove(w)
    return loosers

In [59]:
# load the data to analyse
games_id = np.load("data/subset500/games_id.npy")
all_friendships = np.load("data/subset500/friendships.npy", allow_pickle=True)
verbose = False

data_overall = np.zeros(shape = (2,2))
data_engaged = np.zeros(shape = (3,2))

treatments_overall = ["single", "engaged"]
treatments_engaged = ["betrayer", "betrayed", "neutral"]
outcomes = ["winner", "loser"]

stats_overall = pd.DataFrame(data_overall, index = treatments_overall, columns = outcomes )
stats_engageds = pd.DataFrame(data_engaged, index = treatments_engaged, columns = outcomes )

N = len(games_id)
for i, game_id in enumerate(games_id):
    # reconstruct the obtained data
    data = all_friendships[i]
    years = np.arange(1901, 1901 + data.shape[0] * 0.5, 0.5)
    friendships = pd.DataFrame(data = all_friendships[i], columns = pairs, index = years)
    
    # get outcomes
    winners = get_winners(game_id)
    losers = get_losers(winners)
    
    # get treatment 1 
    engageds = get_engageds(friendships)
    singles = get_singles(engageds)
    
    # get treatment 2
    betrayers, betrayeds = get_betrayers_and_betrayed(friendships)
    neutrals = get_neutrals(betrayers, betrayeds)
    
    # overal statistics
    for w in winners:
        if w in singles: stats_overall.loc["single", "winner"] += 1
        if w in engageds: stats_overall.loc["engaged", "winner"] += 1
    for l in losers:
        if l in singles: stats_overall.loc["single", "loser"] += 1
        if l in engageds: stats_overall.loc["engaged", "loser"] += 1
    
    # statistics about betrayers: 
    for winner in winners: 
        if winner in engageds: 
            if winner in betrayers: stats_engageds.loc["betrayer", "winner"] += 1
            if winner in betrayeds: stats_engageds.loc["betrayed", "winner"] += 1
            if winner in neutrals: stats_engageds.loc["neutral", "winner"] += 1
    for loser in losers: 
        if winner in engageds:
            if loser in betrayers: stats_engageds.loc["betrayer", "loser"] += 1
            if loser in betrayeds: stats_engageds.loc["betrayed", "loser"] += 1
            if loser in neutrals: stats_engageds.loc["neutral", "loser"] += 1
            
    if verbose:
        print("\nGame",i)
        print("Winners: ", winners, " and Losers", losers)
        print("Betrayers: ", betrayers, " and Betrayed", betrayeds)
        print("Neutrals: ", neutrals)

win_ratio = stats.winner / (stats.loser + stats.winner)
stats["win_ratio"] = win_ratio
print("Stats for N =", N)

Stats for N = 500


In [67]:
win_ratio = stats_overall.winner / (stats_overall.loser + stats_overall.winner)
stats_overall["win_ratio"] = win_ratio
print("Statistics over all dataset")
stats_overall

Statistics over all dataset


,winner,loser,win_ratio
single,174.0,1937.0,0.082425
engaged,389.0,1000.0,0.280058


In [68]:
win_ratio = stats_engageds.winner / (stats_engageds.loser + stats_engageds.winner)
stats_engageds["win_ratio"] = win_ratio
print("Statistics over players engaged in a friendship")
stats_engageds

Statistics over players engaged in a friendship


,winner,loser,win_ratio
betrayer,76.0,78.0,0.493506
betrayed,47.0,107.0,0.305195
neutral,267.0,1844.0,0.126480


What can we see here ? 
- among all players that were involved in a broken friendship (either *betrayer* or *betrayed*) the chances of winning go towards the betrayer. Betrayed players have much higher chances of loosing and about 5 times less chances of winning. . .. 
- the neutral players represents the majority of players, however their chances of winning are not much bigger than the chances of win than betrayed people. 

These results make us strongly believe that **betrayals strongly influence the outcome of the game**. 

What can we do next ? 

- select 250 games with  and without betrayals (and all with friendships), then do a matching based on the games properties, and look at what differs once a betrayal happened for the players who were engaged in a friendship !

- quantify agressivity of players towards others, and using the same dataset as before, try to see what happens to a player that was betrayed

# Machine Learning Analysis: can we predict betrayals using game features ? 

In [69]:
import statsmodels.formula.api as smf

features = pd.read_pickle("data/features1/features.pkl")
features

,game_id,has_betrayal,length,outcome,n_aoh_x,n_aoh_y,n_aos_x,n_aos_y,supports_xy,supports_yx
0,96441.0,False,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,96441.0,False,2.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0
2,96441.0,False,3.0,0.0,2.0,0.0,0.0,0.0,1.0,1.0
3,96441.0,False,4.0,0.0,2.0,2.0,1.0,0.0,1.0,1.0
4,96441.0,False,5.0,0.0,1.0,3.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...
4,104120.0,False,5.0,0.0,2.0,1.0,0.0,0.0,1.0,0.0
0,77145.0,True,1.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0
1,77145.0,True,2.0,0.0,2.0,1.0,0.0,0.0,1.0,1.0
2,77145.0,True,3.0,0.0,2.0,1.0,0.0,0.0,1.0,1.0


In [9]:
# train logistic regression

# 1. normalisation 
predictors = ["length", "n_aoh_x", "n_aoh_y","n_aos_x", "n_aos_y", "supports_xy", "supports_yx"]
# predictors = ["length", "n_aoh_x", "n_aoh_y", "n_aos_y",  "supports_yx"]

features = features[predictors + ["outcome"]]

for p in predictors: 
    features[p] = (features[p] - features[p].mean()) / features[p].std()

features

,length,n_aoh_x,n_aoh_y,n_aos_x,n_aos_y,supports_xy,supports_yx,outcome
0,-0.963608,0.262881,1.740934,-0.247537,-0.20004,1.035295,2.229869,0.0
1,-0.727338,1.156678,1.740934,-0.247537,-0.20004,1.035295,2.229869,0.0
2,-0.491069,2.050474,0.126613,-0.247537,-0.20004,0.767275,1.205335,0.0
3,-0.254800,0.262881,0.933773,-0.247537,-0.20004,1.839357,1.205335,0.0
4,-0.018531,1.156678,0.933773,-0.247537,-0.20004,2.375398,1.546846,0.0
5,0.217738,2.050474,1.740934,-0.247537,-0.20004,3.447480,0.863823,0.0
0,-0.963608,1.156678,-0.680547,-0.247537,-0.20004,-1.376890,0.180800,0.0
1,-0.727338,0.262881,0.126613,-0.247537,-0.20004,-1.376890,0.180800,0.0
2,-0.491069,-0.630915,0.126613,-0.247537,-0.20004,0.231233,0.180800,0.0
0,-0.963608,-0.630915,-0.680547,-0.247537,-0.20004,0.231233,0.180800,0.0


In [10]:
formula = "outcome ~ " + ' + '.join(predictors)
formula

'outcome ~ length + n_aoh_x + n_aoh_y + n_aos_x + n_aos_y + supports_xy + supports_yx'

In [11]:
mod = smf.logit(formula = formula, data = features)
res = mod.fit()
res.summary()

         Current function value: inf
         Iterations: 35


/home/arthur/anaconda3/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:1799: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/home/arthur/anaconda3/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:1852: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


LinAlgError: Singular matrix

In [ ]:
tmp = res.predict()
for i, prediction in enumerate(tmp):
    print(prediction, " versus : ", features.iloc[i, -1])



THIS IS OLD ....

# Paired experiment

Paired study over 500 friendships: 250 that didn't ended up in betrayal, and 250 that did. For each friendship, we have
- the game_id
- the players involved : betrayer / betrayeds
- the winners / the losers 

and things we can do the matching upon (*using which outcome ? Maybe 'is one of the player winner'*)
- length of the friendship
- length of the game
- ? average score of the 2 players when they become friends. 
- ? average score .. when they quit being friends.

Then we can attribute to each player a status 'betrayer' or 'betrayed'. 

And finally we could look at the following statistics 
- probability of winning of one of the player.  
- average agressivity of one of the player towards the other. 

## Study design

In order to do this study, we must obtain a new dataset. In the notations, 's' means 'start of friendship', 'e' means 'end of friendship' and 'f' means 'final' (end of the game).

- desired columns: "game_ids", "has_betrayed", "betrayer", "betrayed", "winners", "friendship_length", "game_length", "score_s", "score_e", "score_f", "agressivity_s", "agressivity_e"
- each row is one game ...

Method
1. Construct the dataset
2. Propensity score matching using 'has_betrayed' as outcome
3. Observing effect over the 2 groups: "control" and "treated"

# Potential plots
